#Imports 

In [ ]:
import pandas as pd

In [ ]:
!pip install torchtext==0.6.0 --quiet
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data import Field, BucketIterator, TabularDataset, LabelField
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00


#DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# directory path
# CS779_NLP_COMP/2_Sentiment_Analysis
base_dir = '/content/drive/MyDrive/CS779_NLP_COMP/2_Sentiment_Analysis/'

# CS779_NLP_COMP/2_Sentiment_Analysis/datasets/train.csv
folder_name = 'datasets'

# file_path
file_path = base_dir + folder_name + '/'

In [ ]:
# CS779_NLP_COMP/2_Sentiment_Analysis/datasets/train.csv
df = pd.read_csv(file_path+"train.csv")
df.head()

,text_id,sentence,gold_label
0,r1-0051002,"Cheers,\n\nDennis Nguyen\n416-879-6431",0
1,r1-0020356,May have to wait longer on holidays.,-1
2,r1-0058348,"I drove to vegas may 6th, to get my hair done.",0
3,r1-0080006,"In addition, I eat out often at various restau...",1
4,r1-0000827,Perhaps she was doing us a favor?,0


# Data Pre-processing (Using torchtext and Spacy)

In [ ]:
print("Number of rows: ", df.shape[0])
print("Number of columns: ", df.shape[1])

Number of rows:  92228
Number of columns:  3


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!python -m spacy download en_core_web_sm --quiet


2023-04-14 09:17:26.563896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
nlp_en = spacy.load("en_core_web_sm")

In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 856.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0+cu118
    Uninstalling torch-2.0.0+cu118:
      Successfully uninstalled torch-2.0.0+cu118
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1+cu118
    Uninstalling torchvision-0.15.1+cu118:
      Successfully uninstalled torchvision-0.15.1+cu118
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
ERROR: pip's dependency resolver does not curre

In [ ]:

!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Tokenization: 


In [ ]:
def preprocess_text_en(text):
    # Lowercase 
    text = text.lower()
    # Clean Text
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    if not re.search('[a-zA-Z]', text):
        text = "z"
    doc = nlp_en(text)
    tokens = [token.text for token in doc]
    return tokens


In [ ]:
# Test Run
sample = "This is Sentiment Analysis model"
print(preprocess_text_en(sample))

['i', 'love', 'machine', 'learning']


## Train/ Valid/ Test Split: 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.drop("text_id", axis = 1, inplace = True)

In [ ]:
df.head()

,sentence,gold_label
0,"Cheers,\n\nDennis Nguyen\n416-879-6431",0
1,May have to wait longer on holidays.,-1
2,"I drove to vegas may 6th, to get my hair done.",0
3,"In addition, I eat out often at various restau...",1
4,Perhaps she was doing us a favor?,0


In [ ]:
# Split data into training and testing
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
train_df.to_csv('train.csv', index = False)
test_df.to_csv('test.csv', index = False)

In [ ]:
train_df.head()

,sentence,gold_label
88226,Our son joined us in the X-Child room which we...,0
68999,One concern .,-1
49345,And the LPN Rebecca the others I can say diffr...,-1
35123,"In March, I ordered 7 patio cushion and 4 pill...",0
32949,We in a bit of hurry today so did not browse.,0


In [ ]:
train_df["sentence"][:2]

88226    Our son joined us in the X-Child room which we...
68999                                        One concern .
Name: sentence, dtype: object

In [ ]:
train_df["gold_label"][:2]

88226    0
68999   -1
Name: gold_label, dtype: int64

## Generate vocabulary
https://torchtext.readthedocs.io/en/latest/data.html

In [ ]:
text_field = Field(tokenize=preprocess_text_en,
               lower=True,
               init_token="<sos>",
               eos_token="<eos>")

label_field = LabelField(dtype=torch.float)

train_data, test_data = TabularDataset.splits(
                            path='.', 
                            train='train.csv', 
                            test='test.csv', 
                            format='csv', 
                            skip_header = True,
                            fields=[('sentence', text_field), ('gold_label', label_field)])

# build vocabulary
text_field.build_vocab(train_data,min_freq=1)
label_field.build_vocab(train_data)

In [ ]:
# Define iterator
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iter, test_iter = BucketIterator.splits(
                                    (train_data, test_data), 
                                    batch_size=BATCH_SIZE, 
                                    sort_within_batch=True, 
                                    sort_key=lambda x: len(x.sentence),
                                    device=device)

# Data Pre-processing (Using Sklearn)
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Split data into training and testing
train_df, test_df = train_test_split(df, test_size=0.001, random_state=42)
X_train = train_df["sentence"]
X_train[:5]

In [ ]:
y_train = train_df["gold_label"]
y_train[:5]

### Vectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(test_df["sentence"])
Y_test = test_df["gold_label"]

# References

https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
<br>
https://drive.google.com/file/d/1D8dZAC3QAtAuPKfSb0qmBm1NxPcv4vUq/view
<br>
https://torchtext.readthedocs.io/en/latest/data.html